In [1]:
import pandas as pd
import numpy as np
import torch
import dgl

In [38]:
df_clin = pd.read_table('./Clinical.tsi')
df_clin = df_clin.T
cols = df_clin.iloc[0,:]
df_clin = df_clin.iloc[1:, :]
df_clin.columns = cols
df_clin = df_clin[~df_clin.overall_survival.isna()]
df_clin = df_clin[~df_clin.status.isna()]
df_clin.overall_survival = df_clin.overall_survival.astype('int')
df_clin.status = df_clin.status.astype('int')
df_clin.loc[(df_clin.overall_survival < 2*365) & (df_clin.status == 1), 'label'] = 1
df_clin.loc[(df_clin.overall_survival >= 2*365), 'label'] = 0
df_clin = df_clin[~df_clin.label.isna()]
df_clin

attrib_name,years_to_birth,Tumor_purity,histological_type,gender,radiation_therapy,race,ethnicity,overall_survival,status,overallsurvival,label
TCGA.06.6391,44,0.836,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,45,1,"45,1",1.0
TCGA.19.A6J4,68,0.6736,glioblastomamultiforme(gbm),male,NaN,blackorafricanamerican,nothispanicorlatino,121,1,"121,1",1.0
TCGA.06.1806,47,0.7913,glioblastomamultiforme(gbm),male,yes,white,nothispanicorlatino,466,1,"466,1",1.0
TCGA.06.5408,54,0.8635,untreatedprimary(denovo)gbm,female,yes,white,nothispanicorlatino,357,1,"357,1",1.0
TCGA.06.5410,72,0.3396,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,108,1,"108,1",1.0
...,...,...,...,...,...,...,...,...,...,...,...
TCGA.76.6280,57,0.8776,untreatedprimary(denovo)gbm,male,yes,white,nothispanicorlatino,346,1,"346,1",1.0
TCGA.76.6283,71,0.9779,glioblastomamultiforme(gbm),female,NaN,white,nothispanicorlatino,165,1,"165,1",1.0
TCGA.76.6660,73,0.7707,glioblastomamultiforme(gbm),male,NaN,white,nothispanicorlatino,114,1,"114,1",1.0
TCGA.81.5910,64,0.7913,untreatedprimary(denovo)gbm,male,no,white,NaN,49,1,"49,1",1.0


In [39]:
df_clin = df_clin.reset_index(drop=False)
df_clin

attrib_name,index,years_to_birth,Tumor_purity,histological_type,gender,radiation_therapy,race,ethnicity,overall_survival,status,overallsurvival,label
0,TCGA.06.6391,44,0.836,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,45,1,"45,1",1.0
1,TCGA.19.A6J4,68,0.6736,glioblastomamultiforme(gbm),male,NaN,blackorafricanamerican,nothispanicorlatino,121,1,"121,1",1.0
2,TCGA.06.1806,47,0.7913,glioblastomamultiforme(gbm),male,yes,white,nothispanicorlatino,466,1,"466,1",1.0
3,TCGA.06.5408,54,0.8635,untreatedprimary(denovo)gbm,female,yes,white,nothispanicorlatino,357,1,"357,1",1.0
4,TCGA.06.5410,72,0.3396,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,108,1,"108,1",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
489,TCGA.76.6280,57,0.8776,untreatedprimary(denovo)gbm,male,yes,white,nothispanicorlatino,346,1,"346,1",1.0
490,TCGA.76.6283,71,0.9779,glioblastomamultiforme(gbm),female,NaN,white,nothispanicorlatino,165,1,"165,1",1.0
491,TCGA.76.6660,73,0.7707,glioblastomamultiforme(gbm),male,NaN,white,nothispanicorlatino,114,1,"114,1",1.0
492,TCGA.81.5910,64,0.7913,untreatedprimary(denovo)gbm,male,no,white,NaN,49,1,"49,1",1.0


In [40]:
df_clin.columns

Index(['index', 'years_to_birth', 'Tumor_purity', 'histological_type',
       'gender', 'radiation_therapy', 'race', 'ethnicity', 'overall_survival',
       'status', 'overallsurvival', 'label'],
      dtype='object', name='attrib_name')

In [16]:
df_clin.columns = ['patient_id', 'feature_age', 'feature_tumor_purity', 'feature_histological_type',
                   'feature_gender', 'feature_radiation_therapy', 'feature_race', 'feature_ethnicity',
                   'overall_survival', 'status', 'overallsurvival', 'label']
df_clin

,patient_id,feature_age,feature_tumor_purity,feature_histological_type,feature_gender,feature_radiation_therapy,feature_race,feature_ethnicity,overall_survival,status,overallsurvival,label
0,TCGA.06.6391,44,0.836,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,45,1,"45,1",1.0
1,TCGA.19.A6J4,68,0.6736,glioblastomamultiforme(gbm),male,NaN,blackorafricanamerican,nothispanicorlatino,121,1,"121,1",1.0
2,TCGA.06.1806,47,0.7913,glioblastomamultiforme(gbm),male,yes,white,nothispanicorlatino,466,1,"466,1",1.0
3,TCGA.06.5408,54,0.8635,untreatedprimary(denovo)gbm,female,yes,white,nothispanicorlatino,357,1,"357,1",1.0
4,TCGA.06.5410,72,0.3396,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,108,1,"108,1",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
489,TCGA.76.6280,57,0.8776,untreatedprimary(denovo)gbm,male,yes,white,nothispanicorlatino,346,1,"346,1",1.0
490,TCGA.76.6283,71,0.9779,glioblastomamultiforme(gbm),female,NaN,white,nothispanicorlatino,165,1,"165,1",1.0
491,TCGA.76.6660,73,0.7707,glioblastomamultiforme(gbm),male,NaN,white,nothispanicorlatino,114,1,"114,1",1.0
492,TCGA.81.5910,64,0.7913,untreatedprimary(denovo)gbm,male,no,white,NaN,49,1,"49,1",1.0


In [20]:
df_clin.to_csv('./clinical.csv.gz', index=False, compression='gzip')
df_clin = pd.read_csv('./clinical.csv.gz', compression='gzip')

In [21]:
df_clin

,patient_id,feature_age,feature_tumor_purity,feature_histological_type,feature_gender,feature_radiation_therapy,feature_race,feature_ethnicity,overall_survival,status,overallsurvival,label
0,TCGA.06.6391,44,0.8360,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,45,1,"45,1",1.0
1,TCGA.19.A6J4,68,0.6736,glioblastomamultiforme(gbm),male,NaN,blackorafricanamerican,nothispanicorlatino,121,1,"121,1",1.0
2,TCGA.06.1806,47,0.7913,glioblastomamultiforme(gbm),male,yes,white,nothispanicorlatino,466,1,"466,1",1.0
3,TCGA.06.5408,54,0.8635,untreatedprimary(denovo)gbm,female,yes,white,nothispanicorlatino,357,1,"357,1",1.0
4,TCGA.06.5410,72,0.3396,untreatedprimary(denovo)gbm,female,no,white,nothispanicorlatino,108,1,"108,1",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
489,TCGA.76.6280,57,0.8776,untreatedprimary(denovo)gbm,male,yes,white,nothispanicorlatino,346,1,"346,1",1.0
490,TCGA.76.6283,71,0.9779,glioblastomamultiforme(gbm),female,NaN,white,nothispanicorlatino,165,1,"165,1",1.0
491,TCGA.76.6660,73,0.7707,glioblastomamultiforme(gbm),male,NaN,white,nothispanicorlatino,114,1,"114,1",1.0
492,TCGA.81.5910,64,0.7913,untreatedprimary(denovo)gbm,male,no,white,NaN,49,1,"49,1",1.0


In [30]:
cols = ['patient_id', 'label']
for col in df_clin.columns.values:
    if col.startswith('feature'):
        cols.append(col)
df_clin = df_clin[cols]

In [36]:
df_clin = pd.concat([df_clin.iloc[:, :2],pd.get_dummies(df_clin.iloc[:, 2:])], axis=1)
df_clin

,patient_id,label,feature_age,feature_tumor_purity,feature_histological_type_glioblastomamultiforme(gbm),feature_histological_type_treatedprimarygbm,feature_histological_type_untreatedprimary(denovo)gbm,feature_gender_female,feature_gender_male,feature_radiation_therapy_no,feature_radiation_therapy_yes,feature_race_asian,feature_race_blackorafricanamerican,feature_race_white,feature_ethnicity_hispanicorlatino,feature_ethnicity_nothispanicorlatino
0,TCGA.06.6391,1.0,44,0.8360,0,0,1,1,0,1,0,0,0,1,0,1
1,TCGA.19.A6J4,1.0,68,0.6736,1,0,0,0,1,0,0,0,1,0,0,1
2,TCGA.06.1806,1.0,47,0.7913,1,0,0,0,1,0,1,0,0,1,0,1
3,TCGA.06.5408,1.0,54,0.8635,0,0,1,1,0,0,1,0,0,1,0,1
4,TCGA.06.5410,1.0,72,0.3396,0,0,1,1,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,TCGA.76.6280,1.0,57,0.8776,0,0,1,0,1,0,1,0,0,1,0,1
490,TCGA.76.6283,1.0,71,0.9779,1,0,0,1,0,0,0,0,0,1,0,1
491,TCGA.76.6660,1.0,73,0.7707,1,0,0,0,1,0,0,0,0,1,0,1
492,TCGA.81.5910,1.0,64,0.7913,0,0,1,0,1,1,0,0,0,1,0,0


In [37]:
df_clin.label.value_counts()

1.0    386
0.0    108
Name: label, dtype: int64

In [49]:
df = pd.read_table('./GBM.cnv.tsv.gz', compression='gzip')
df = df.rename(columns={'attrib_name': 'gene'})
df.to_csv('./GBM.cnv.csv.gz', compression='gzip', index=False)

In [105]:
pd.read_csv('./GBM.cnv.csv.gz', compression='gzip')

,gene,TCGA.12.3652,TCGA.19.0960,TCGA.12.1099,TCGA.06.0130,TCGA.02.0028,TCGA.28.1750,TCGA.32.2616,TCGA.06.2564,TCGA.16.1056,...,TCGA.12.1091,TCGA.06.2570,TCGA.32.1986,TCGA.19.2629,TCGA.14.1795,TCGA.27.2524,TCGA.02.0074,TCGA.02.0057,TCGA.06.0876,TCGA.28.1747
0,55210,-0.052,-0.006,0.259,0.123,-0.887,-0.493,0.102,0.042,-0.748,...,0.081,0.010,0.037,-0.201,0.032,-0.858,0.029,-0.014,-0.034,0.032
1,54991,-0.052,-0.006,0.259,0.123,-0.887,-0.493,0.102,0.042,-0.748,...,0.081,0.010,0.037,-0.201,0.032,-0.858,0.029,-0.014,-0.034,0.032
2,1855,-0.052,-0.006,0.259,0.123,-0.887,-0.493,0.102,0.042,-0.748,...,0.081,0.010,0.037,-0.201,0.032,-0.858,0.029,-0.014,-0.034,0.032
3,2563,-0.052,-0.006,0.259,0.123,-0.887,-0.493,0.102,0.042,-0.748,...,0.081,0.010,0.037,-0.201,0.032,-0.858,0.029,-0.014,-0.034,0.032
4,2782,-0.052,-0.006,0.259,0.123,-0.887,-0.493,0.102,0.042,-0.748,...,0.081,0.010,0.037,-0.201,0.032,-0.858,0.029,-0.014,-0.034,0.032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8569,29781,-0.075,-0.027,0.310,0.154,0.063,0.046,0.039,0.029,-0.740,...,-0.912,-0.008,-0.526,-0.481,0.029,-0.012,0.243,-0.135,0.000,-0.005
8570,6305,-0.075,-0.027,0.310,0.154,0.063,0.046,0.039,0.029,-0.740,...,-0.912,-0.008,-0.526,-0.481,0.029,-0.012,0.243,-0.135,0.000,-0.005
8571,9997,-0.075,-0.027,0.310,0.154,0.063,0.046,0.039,0.029,-0.740,...,-0.912,-0.008,-0.526,-0.481,0.029,-0.012,0.243,-0.135,0.000,-0.005
8572,80305,-0.075,-0.027,0.310,0.154,0.063,0.046,0.039,0.029,-0.740,...,-0.912,-0.008,-0.526,-0.481,0.029,-0.012,0.243,-0.135,0.000,-0.005
